In [3]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import InceptionV3

In [4]:
# Set the paths to your data folders
train_data_dir = 'data/train'
test_data_dir = 'data/test'

# Define image dimensions and batch size
img_width, img_height = 150, 150
batch_size = 32

# Data preprocessing and augmentation
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   validation_split=0.2)  # Add validation split

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                    target_size=(img_width, img_height),
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    color_mode='rgb',  # Ensure images are loaded in RGB format
                                                    subset='training')  # Specify 'training' subset

validation_generator = train_datagen.flow_from_directory(train_data_dir,
                                                         target_size=(img_width, img_height),
                                                         batch_size=batch_size,
                                                         class_mode='categorical',
                                                         color_mode='rgb',  # Ensure images are loaded in RGB format
                                                         subset='validation')  # Specify 'validation' subset

test_generator = test_datagen.flow_from_directory(test_data_dir,
                                                  target_size=(img_width, img_height),
                                                  batch_size=batch_size,
                                                  class_mode='categorical',
                                                  color_mode='rgb')  # Ensure images are loaded in RGB format



Found 96 images belonging to 3 classes.
Found 24 images belonging to 3 classes.
Found 60 images belonging to 3 classes.


In [5]:
# Define the CNN model
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(3, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

In [6]:
# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [7]:
# Train the model
model.fit(train_generator,
          steps_per_epoch=len(train_generator),
          epochs=10,
          validation_data=validation_generator,
          validation_steps=len(validation_generator))

Epoch 1/10


E:\anacondapython\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


3/3 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.3099 - loss: 2.4708 - val_accuracy: 0.6667 - val_loss: 1.6877
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00


E:\anacondapython\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 374ms/step - accuracy: 0.7318 - loss: 0.7584 - val_accuracy: 0.8333 - val_loss: 0.3029
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 354ms/step - accuracy: 0.9062 - loss: 0.2982 - val_accuracy: 0.8750 - val_loss: 0.1579
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 369ms/step - accuracy: 0.9831 - loss: 0.0521 - val_accuracy: 0.9583 - val_loss: 0.1390
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 9/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 390ms/step - accuracy: 1.0000 - loss: 0.0194 - val_accuracy: 0.9167 - val_loss: 0.2134
Epoch 10/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.0000e+00 

In [8]:
# Save the trained model
model.save('image_classifier_model_with_inceptionv3.h5')


In [9]:
# Evaluate the model
print("Model evaluation on test data:")
model.evaluate(test_generator, steps=len(test_generator))

Model evaluation on test data:
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step - accuracy: 0.9889 - loss: 0.0634


[0.06993471086025238, 0.9833333492279053]

In [12]:
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model('image_classifier_model_with_inceptionv3.h5')

# Function to predict the class of an input image
def predict_class(image_path):
    img = image.load_img(image_path, target_size=(img_width, img_height))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.  # Rescale pixel values

    # Make prediction
    predictions = model.predict(img_array)

    # Interpret predictions
    class_labels = ['cars', 'dogs', 'mountains']  # Define your class labels
    predicted_class_index = np.argmax(predictions)
    predicted_class_label = class_labels[predicted_class_index]

    return predicted_class_label, predictions[0][predicted_class_index]

# Test with an input image
path1 = 'C:\\Users\\madda\\Downloads\\insta_project\\data\\test\\cars\\6.jpg'
predicted_class, confidence = predict_class(path1)
print("Predicted class:", predicted_class)
print("Confidence:", confidence)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Predicted class: cars
Confidence: 0.99998605
